<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
这是<a href="http://mng.bz/orYv">从零开始构建一个大语言模型</a>这本书的补充代码。 作者 <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>代码仓库: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# 理解嵌入(Embedding Layers)层和线性层(Linear Layers)的区别

- PyTorch中嵌入层与线性层的作用相同，使用嵌入层的原因是为了提高计算效率
- 我们将使用PyTorch中的代码示例一步一步地查看这种关系

In [1]:
import torch

print("PyTorch version:", torch.__version__)

PyTorch version: 2.3.1


<br>
&nbsp;

## 使用 nn.Embedding

In [2]:
# Suppose we have the following 3 training examples,
# which may represent token IDs in a LLM context
# 假设我们有以下三个训练样本，它们可能代表LLM上下文中的token ID
idx = torch.tensor([2, 3, 1])

# 嵌入矩阵的行数可以由获得最大token ID + 1得到。
# 如果最大token ID是3，那么我们需要4行，可能的token ID 0, 1, 2, 3
num_idx = max(idx)+1

# 所需的嵌入维度是一个超参数
out_dim = 5

- Let's implement a simple embedding layer:

In [3]:
# 我们使用随机种子，以便在嵌入层中的权重被初始化为小随机值
torch.manual_seed(123)

embedding = torch.nn.Embedding(num_idx, out_dim)

我们可以选择查看嵌入权重：

In [4]:
embedding.weight

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.3035, -0.5880,  1.5810],
        [ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015],
        [ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096, -0.4076,  0.7953]], requires_grad=True)

- 我们可以用embedding层来获得训练样本ID为1的向量表示：

In [5]:
embedding(torch.tensor([1]))

tensor([[ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015]],
       grad_fn=<EmbeddingBackward0>)

- 下面是发生在幕后发生的可视化：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/embeddings-and-linear-layers/1.png" width="400px">

- 同样地，我们可以使用嵌入层来获得训练样本ID 2的向量表示：

In [6]:
embedding(torch.tensor([2]))

tensor([[ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315]],
       grad_fn=<EmbeddingBackward0>)

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/embeddings-and-linear-layers/2.png" width="400px">

- 现在，让我们转换之前定义的所有训练样本：

In [7]:
idx = torch.tensor([2, 3, 1])
embedding(idx)

tensor([[ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096, -0.4076,  0.7953],
        [ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015]],
       grad_fn=<EmbeddingBackward0>)

- 在底层，依然是查找概念：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/embeddings-and-linear-layers/3.png" width="450px">

<br>
&nbsp;

## 使用 nn.Linear

- 现在，我们将展示embedding层在PyTorch中是否可以实现与`nn.Linear`层相同的功能，即在独热编码表示上
- 首先，我们将Token ID转换为独热表示：

In [8]:
onehot = torch.nn.functional.one_hot(idx)
onehot

tensor([[0, 0, 1, 0],
        [0, 0, 0, 1],
        [0, 1, 0, 0]])

- 下一步，我们初始化一个`Linear`层，它执行矩阵乘法$X W^\top$：

In [9]:
torch.manual_seed(123)
linear = torch.nn.Linear(num_idx, out_dim, bias=False)
linear.weight

Parameter containing:
tensor([[-0.2039,  0.0166, -0.2483,  0.1886],
        [-0.4260,  0.3665, -0.3634, -0.3975],
        [-0.3159,  0.2264, -0.1847,  0.1871],
        [-0.4244, -0.3034, -0.1836, -0.0983],
        [-0.3814,  0.3274, -0.1179,  0.1605]], requires_grad=True)

- 注意，PyTorch中的线性层也初始化了一些小的随机权重；为了与上面的`Embedding`层直接比较它，我们必须使用相同的小随机权重，这就是为什么我们重新赋值它们的原因。

In [10]:
linear.weight = torch.nn.Parameter(embedding.weight.T)

- 现在我们可以在输入的独热编码表示上使用线性层:

In [11]:
linear(onehot.float())

tensor([[ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096, -0.4076,  0.7953],
        [ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015]], grad_fn=<MmBackward0>)

正如我们所见，这与使用嵌入层得到的结果完全相同：

In [12]:
embedding(idx)

tensor([[ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096, -0.4076,  0.7953],
        [ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015]],
       grad_fn=<EmbeddingBackward0>)

- 下面是计算第一个训练示例Token ID的示意：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/embeddings-and-linear-layers/4.png" width="450px">

- 第二个训练示例的token ID:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/embeddings-and-linear-layers/5.png" width="450px">

- 由于每个独热编码行中的除了一个索引外其他的都是0（根据设计），因此这个矩阵乘法实际上与查找独热编码元素的效果相同
- 在独热编码上使用矩阵乘法相当于嵌入层查找，但如果我们使用大型嵌入矩阵，效率可能会很低，因为有很多浪费的零乘法